# Decisionn Trees
![](picture/01.png)

Check if every item in the dataset is in the same class:

    if so return the class label
    Else
        find the best feature to split the data
        split the dataset
        create a branch node
             for each split
                 call createBranch and add the result to the branch node
             retuen branch node
             
Please note the recursive nature of createBranch. It calls itself in the second-to-last line.

**Note:** some decision trees make a binary split of the data,but we won't do this. We'll follow the ID3 algorithm.

### ID3

Entropy is defined as the expected value of the information. First, we need to define information. If you’re classifying something that can take on multiple values, the information for symbol xi is defined as

$l(x_i) = log_{2}p(x_i)$

where p(xi) is the probability of choosing this class.

To calculate entropy, you need the expected value of all the information of all pos-
sible values of our class. This is given by

$H = - \sum_{i=1}^{n}p(x_i)log_{2}p(x_i)$

where n is the number of classes.

Note: click [here](https://blog.csdn.net/acdreamers/article/details/44661149) to ID3

### DataSet

See the data in table 3.1. It contains five animals pulled from the sea and asks if they can survive without coming to the surface and if they have flippers. We would like to classify these animals into two classes: fish and not fish. Now we want to decide whether we should split the data based on the first feature or the second feature. To answer this question, we need some quantitative way of determining how to split the data. We’ll discuss that next.

![](picture/02.png)

### Let's look how to compute SannonENT

If we target learnning "is Fish?".

The table 3.1 total examples is 5, and 2 "yes",3 "no". So,we compute shannonENT as follows

$H = - \frac{3}{5} log_{2}\frac{3}{5} - \frac{2}{5} log_{2} \frac{2}{5} = 0.9709505944546686$


### 1.Function to calculate the Shannon entropy of a dataset

In [1]:
from math import log

def calcShannonEnt(dataSet):
    """
    parameter-- dataSet,it's a dictionary
    
    return shannon Ent
    """
    numEntrise = len(dataSet)
    labelCounts = {}
    for featVec in dataSet:
        currentLabel = featVec[-1]
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel] = 0
            labelCounts[currentLabel] += 1
        else:
            labelCounts[currentLabel] += 1
    shannonEnt = 0.
#     print(labelCounts)
    for key in labelCounts:
        # compute shannon entropy 
        prob = float(labelCounts[key]) /  numEntrise
        shannonEnt -= prob * log(prob,2)
    return shannonEnt

In [2]:
def createDataSet():
    """
    create data set with table 3.1 Marine animal data
    Note: 1 = Yes ,0= No in fetures.
    returns-- data set and labels
    """
    dataSet = [[1,1,'yes'],
              [1,1,"yes"],
              [1,0,"no"],
              [0,1,"no"],
              [0,1,"no"]]
    labels = ['no surfacing','flippers']
    return dataSet,labels

Now, we can try compute shannonENT

In [3]:
dataSet,labels = createDataSet()
shannonENT = calcShannonEnt(dataSet)
print(shannonENT)

0.9709505944546686


### 2.Splitting the dataset
You just saw how to measure the amount of disorder in a dataset. For our classifier algorithm to work, you need to measure the entropy, split the dataset, measure the entropy on the split sets, and see if splitting it was the right thing to do. You’ll do this for all of our features to determine the best feature to split on. Think of it as a two- dimensional plot of some data. You want to draw a line to separate one class from another. Should you do this on the X-axis or the Y-axis? The answer is what you’re try- ing to find out here.

In [4]:
def splitDataSet(dataSet,axis,value):
    """
    Dataset splitting on given feture
    
    returns : retDataSet
    """
    
    retDataSet = []
    for featVec in dataSet:
        # start split dataset
        if featVec[axis] == value: # if value of featVec[axis] equal target value, then  append to retDataSet
            reducedFeatVec = featVec[:axis]
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet

In [5]:
dataSet,labels = createDataSet()
print(dataSet)
retDataSet = splitDataSet(dataSet,0,1)
print(retDataSet)

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]
[[1, 'yes'], [1, 'yes'], [0, 'no']]


You’re now going to combine the Shannon entropy calculation and the splitDataSet() function to cycle through the dataset and decide which feature is the best to split on. Using the entropy calculation tells you which split best organizes your data.

### 3.Choosing the best feature to split on

在决策树的每一个非叶子结点划分之前，先计算每一个属性所带来的信息增益，选择最大信息增益的属性来划
分，因为信息增益越大，区分样本的能力就越强，越具有代表性，很显然这是一种自顶向下的贪心策略。

In [6]:
def chooseBestFeatureToSplit(dataSet):
    """
    choose the best feture to spilt
    
    return:
        bestFeture
    """
    # get Number of index in every feature list.Note: every feature list must be same.
    numFeatures = len(dataSet[0]) -1 
    # compute "is Fish" feature  base entropy,like example in "dataSet cell".
    baseEntropy = calcShannonEnt(dataSet)
    # set best InfoGain, infoGain is Swedish language~,initialize value equal 0
    # and set best feature. initialize value equal -1
    bestInfoGain = 0.; bestFeature = -1
    for i in range(numFeatures):
        featList = [example[i] for example in dataSet] #get every feature list
        uniqueVals = set(featList) # unique feature list
        newEntropy = 0.
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet,i,value)
            
            # follows code about 2 lines, compute shannonEnt.
            prob = len(subDataSet) / float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataSet)
            
        infoGain = baseEntropy - newEntropy # comput infoGain
        if (infoGain > bestInfoGain): # update infoGain
            bestInfoGain = infoGain
            bestFeature = i  # update best feature
    return bestFeature

In [7]:
dataSet,labels = createDataSet()
print(dataSet)
bestFeature = chooseBestFeatureToSplit(dataSet)
print("Best Feature index is ",bestFeature)

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]
Best Feature index is  0


So,the Best Feature is "Can survive without coming to surface."


You’ll stop under the following conditions: you run out of attributes on which to split or all the instances in a branch are the same class. If all instances have the same class, then you’ll create a leaf node, or terminating block. Any data that reaches this leaf node is deemed to belong to the class of that leaf node. 
![](picture/03.png)

### Befor we create the Decision Tree,we need learning decision tree's logist.

Let's look what is decision tree's logist.

We using this example dataset

![](picture/04.png)


- step 1: we need compute Entropy(Fish) = $- \frac{3}{5} log_{2}\frac{3}{5} - \frac{2}{5} log_{2} \frac{2}{5}=0.9709505944546686$
- step 2: comput IG
    - No surfacing:
        - 1:[is Fish] yes,yes,no 
        - 0:[is Fish] no,no
        - Entropy(1) = $- \frac{2}{3} log_{2}\frac{2}{3} - \frac{1}{3} log_{2} \frac{1}{3}=0.918296 $
        - Entropy(0) = $- 0 - \frac{1}{2} log_{2} \frac{1}{2}=0.5 $
        - Entropy(No surfacing | Fish) = $\frac{3}{5}\times 0.918296 + \frac{2}{5} \times 0.5 = 0.750977$
        - $\frac{3}{5},\frac{2}{5}$: in total dataset
        - IG(No surfacing | Fish) = 0.9709505944546686 - 0.750977 = 0.219973
    - Flippers:
        - 1:[is Fish] yes,yes,no,no
        - 0:[is Fish] no
        - Entropy(1) = $- \frac{2}{4} log_{2}\frac{2}{4} - \frac{2}{4} log_{2} \frac{2}{4}=1.0 $
        - Entropy(0) = $- 0 - \frac{1}{2} log_{2} \frac{1}{1}=0 $
        - Entropy(Flippers | Fish) = $\frac{4}{5}\times 1 + \frac{1}{5} \times 0 = 0.8$
        - $\frac{1}{5},\frac{4}{5}$: in total dataset
        - IG(Flippers | Fish) = 0.9709505944546686 - 0.8 = 0.17095
        
- step 3: choose best IG(max IG)
    - so,we choose feature "No surfacing".
- step 4: using feature "No surfacing" to split dataset.
- step 5: repeat like setp 1-3
    
**Last Note:**
 - if best feature can't split dataset to  have same class label, then ,can using other feature to split dataset.
     - [[1, 1, 'yes'], [1, 0, 'yes'], [1, 0, 'no'], [0, 1, 'yes'], [0, 1, 'no']]
 
 - if case have no more features to split, but we also can't split dataset to have same class label, then we use "majority vote."
     - [[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'yes'], [0, 1, 'no']]
 

In [8]:
import operator
def majorityCnt(classList):
    classCount = {}
    for vote in classList:
        if vote not in classCount.keys(): 
            classCount[vote] = 0
        else:
            classCount[vote] += 1
    sortedClassCount = sorted(classCount.items(),key=operator.itemgetter(1),reverse=True)
    print(sortedClassCount)
    return sortedClassCount[0][0]

In [9]:
majorityCnt(['yes','yes','yes','no'])

[('yes', 2), ('no', 0)]


'yes'

### 4.Tree-building code

The first stopping condition is that if all the class labels are the same, then you return this label.

The second stopping condition is the case when there are no more features to split.

In [10]:
def createTree(dataSet,labels):
    # get class list: ["yes","yes"..] in last feature.
    classList = [example[-1] for example in dataSet]
    
    # define first stopping condition.
    if classList.count(classList[0]) == len(classList):
        
        return classList[0]
    
    # define second stopping condition.
    if len(dataSet[0]) == 1:
        
        return  majorityCnt(classList)
    
    # get best Feature.
    bestFeat = chooseBestFeatureToSplit(dataSet)
    
    # using best feture to get label in labels.
    bestFeatLabel = labels[bestFeat]
    
    # create result tree.
    myTree = {bestFeatLabel:{}}
    
    del (labels[bestFeat])
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues) # get unique values to split dataset.
    for value in uniqueVals:
        subLabels = labels[:] # Get the remaining tables.
        # start Recursive.
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet,bestFeat,value),subLabels)
    return myTree

In [11]:
dataSet,labels = createDataSet()
print(dataSet)
print(labels)
myTree = createTree(dataSet,labels)
print(myTree)

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]
['no surfacing', 'flippers']
{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}


majority vote.

In [12]:
dataSet = [[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'yes'], [0, 1, 'no']]
labels = ['no surfacing', 'flippers']
myTree = createTree(dataSet,labels)
print(myTree)

[('yes', 0), ('no', 0)]
{'flippers': {0: 'no', 1: {'no surfacing': {0: 'yes', 1: 'yes'}}}}


best feature can't split dataset 

In [13]:
dataSet = [[1, 1, 'yes'], [1, 0, 'yes'], [1, 0, 'no'], [0, 1, 'yes'], [0, 1, 'no']]
labels = ['no surfacing', 'flippers']
myTree = createTree(dataSet,labels)
print(myTree)

[('yes', 0), ('no', 0)]
{'no surfacing': {0: {'flippers': {'no': 'no', 'yes': 'yes'}}, 1: {'flippers': {0: 'yes', 1: 'yes'}}}}


### 5.using decision trees to predict contact lens type

The Lenses dataset3 is one of the more famous datasets. It’s a number of observations based on patients’ eye conditions and the type of contact lenses the doctor prescribed. The classes are hard, soft, and no contact lenses. The data is from the UCI database repository and is modified slightly so that it can be displayed easier.

In [26]:
def loadingDataSet():
    """
    Implement predict contact lens
    returns:
        lenesTree
    """
    path = "data_set/lenses.txt"
    fr = open(path)
    lenses = [inst.strip().split('\t') for inst in fr.readlines() ]
    lenesLabels = ['age','prescript','astigmatic','tearRate']
    lenesTree = createTree(lenses,lenesLabels)
    return lenesTree

In [27]:
lenesTree = loadingDataSet()
print(lenesTree)

{'tearRate': {'normal': {'astigmatic': {'no': {'age': {'pre': 'soft', 'young': 'soft', 'presbyopic': {'prescript': {'myope': 'no lenses', 'hyper': 'soft'}}}}, 'yes': {'prescript': {'myope': 'hard', 'hyper': {'age': {'pre': 'no lenses', 'young': 'hard', 'presbyopic': 'no lenses'}}}}}}, 'reduced': 'no lenses'}}


### 6.Summary

The contact lens data showed that decision trees can try too hard and overfit a dataset. This overfitting can be removed by pruning the decision tree, combining adjacent leaf nodes that don’t provide a large amount of information gain.

There are other decision tree–generating algorithms. The most popular are C4.5 and CART. 